In [73]:
from __future__ import division
import requests
import collections
import json
import psycopg2
import time

In [65]:
def get_next_n(conn, limit=0, offset=0):
    cur = conn.cursor()
    cur.execute("SET TIME ZONE 'UTC';")
    
    query = """
            SELECT id
            FROM reports
            ORDER BY id ASC
            LIMIT {}
            OFFSET {}
            """.format(limit, offset)
    cur_report = conn.cursor()    
    cur_report.execute(query)
    
    i = 0
    for bug in cur_report:
        bug_id = str(bug[0])
        #format to https://bugzilla.mozilla.org/rest/bug/707428/comment
        website = 'https://bugzilla.mozilla.org/rest/bug/{}/comment'.format(bug_id)
        #json_content = bson.json_util.loads(requests.get(website).content)
        json_content = json.JSONDecoder().decode(requests.get(website).content)
        bug_date = json_content['bugs'][bug_id]['comments'][0]['creation_time']
        bug_desc = json_content['bugs'][bug_id]['comments'][0]['raw_text']

        #cur.execute("INSERT INTO foo VALUES (%s)", ["bar"])
        cur.execute(
            """
            INSERT INTO descriptions
            VALUES (%s, to_timestamp(%s, 'YYYY-MM-DD HH24:MI:SS') - interval '9 h', %s)
            """, 
            [bug_id, bug_date, bug_desc]
        )
        
        i += 1
        if i % 100 == 0:
            print "inserted {} rows".format(i)
    
    conn.commit()

In [74]:
try:
    conn = psycopg2.connect("dbname='bugs' user='lucka' host='localhost'")
except:
    print "Unable to connect to the database"
    
cur = conn.cursor()
cur.execute("""
    CREATE TABLE IF NOT EXISTS descriptions (
    id            bigint NOT NULL,
    when_created  timestamp,
    descr         text
    );
""")
conn.commit()

for offset in xrange(0, 100000, 5000):
    get_next_n(conn, limit=5000, offset=offset)
    time.sleep(1)

conn.close()